In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import numpy as np
from scipy import stats
from scipy.stats import norm
import seaborn as sns

In [6]:
# Import data
df = pd.read_csv(Path('../resources/regressiondata_2020.csv'))
df.head()

,Unnamed: 0,zpid,latestPrice,numOfBathrooms,livingAreaSqFt,numOfBedrooms,avgSchoolRating,numOfStories,MedianStudentsPerTeacher,numOfHighSchools,longitude,numOfPrimarySchools,zipcode,propertyTaxRate,latest_saledate,latest_salemonth,latest_saleyear
0,1,120900430,295000.0,2.0,1768.0,4,2.666667,1,14,1,-97.661697,1,78660,1.98,2020-10-13,10,2020
1,5,2080105342,309045.0,2.0,1446.0,3,4.000000,1,14,1,-97.656181,1,78660,1.98,2020-08-05,8,2020
2,6,241932337,315000.0,3.0,2432.0,4,3.666667,2,12,1,-97.643394,1,78660,1.98,2020-06-11,6,2020
3,10,241932327,279900.0,2.0,1580.0,3,3.666667,1,12,1,-97.643288,1,78660,1.98,2020-08-28,8,2020
4,12,69808966,239900.0,2.0,1762.0,4,3.333333,1,14,1,-97.623436,1,78617,1.98,2020-09-05,9,2020


In [7]:
df.columns

Index(['Unnamed: 0', 'zpid', 'latestPrice', 'numOfBathrooms', 'livingAreaSqFt',
       'numOfBedrooms', 'avgSchoolRating', 'numOfStories',
       'MedianStudentsPerTeacher', 'numOfHighSchools', 'longitude',
       'numOfPrimarySchools', 'zipcode', 'propertyTaxRate', 'latest_saledate',
       'latest_salemonth', 'latest_saleyear'],
      dtype='object')

In [8]:
# Save 'zpid'
id_df = df['zpid']

# Drop 'zpid' column
df.drop(["zpid","latest_saledate","Unnamed: 0",'latest_salemonth','latest_saleyear'], axis = 1, inplace = True)

# Check data size after dropping the 'Id' variable
print("\nData size: {} ".format(df.shape)) 


Data size: (5412, 12) 


In [9]:
# Create features and target
y = df["latestPrice"]
X = df.drop(columns=['latestPrice'])

In [10]:
# Split data into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(4059, 11)

In [11]:
# Create DMatrices

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [12]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# "Learn" the mean from the training data
mean_train = np.mean(y_train)

# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train

# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))

Baseline MAE is 246150.68


In [13]:
# Create parameters dictionary
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

In [14]:
params['eval_metric'] = "mae"

num_boost_round = 999

In [15]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[0]	Test-mae:383155
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:272743
[2]	Test-mae:206196
[3]	Test-mae:167306
[4]	Test-mae:146528
[5]	Test-mae:136425
[6]	Test-mae:131967
[7]	Test-mae:130426
[8]	Test-mae:130417
[9]	Test-mae:131151
[10]	Test-mae:131748
[11]	Test-mae:131534
[12]	Test-mae:131545
[13]	Test-mae:131832
[14]	Test-mae:131422
[15]	Test-mae:131651
[16]	Test-mae:131765
[17]	Test-mae:130859
[18]	Test-mae:130141
[19]	Test-mae:129831
[20]	Test-mae:129751
[21]	Test-mae:129908
[22]	Test-mae:129966
[23]	Test-mae:130297
[24]	Test-mae:130008
[25]	Test-mae:130019
[26]	Test-mae:130092
[27]	Test-mae:129555
[28]	Test-mae:129394
[29]	Test-mae:129667
[30]	Test-mae:129445
[31]	Test-mae:129352
[32]	Test-mae:129306
[33]	Test-mae:129107
[34]	Test-mae:128785
[35]	Test-mae:128857
[36]	Test-mae:128924
[37]	Test-mae:128854
[38]	Test-mae:128782
[39]	Test-mae:128802
[40]	Test-mae:128968
[41]	Test-mae:128898
[42]	Test-mae:128860
[43]	Test-mae:128622
[44]	Test-mae:128752
[45]	Test

In [16]:
# Get cross validation score with current params
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,388802.806250,2019.679006,389883.168750,9967.839034
1,278543.956250,1419.380050,281148.075000,9580.257574
2,206582.928125,970.726888,214269.025000,9795.424402
3,161084.665625,1173.736398,174227.912500,8489.693611
4,134349.137500,1652.085752,152468.768750,7965.405090
5,119233.543750,1570.250371,141414.862500,7903.448449
6,110268.959375,1758.787065,136491.067187,8494.600787
7,105042.714063,1730.933774,133934.860938,8204.464424
8,101461.800000,1704.931264,132631.559375,8177.447809
9,98779.329688,1338.945352,131760.510938,8116.975105


In [ ]:
cv_results['test-mae-mean'].min()

In [ ]:
# You can try wider intervals with a larger step between
# each value and then narrow it down. Here after several
# iteration I found that the optimal value was in the
# following ranges.
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(4,12)
    for min_child_weight in range(1,10)
]

In [ ]:
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
# Update parameters with best found parameters
params['max_depth'] = 7
params['min_child_weight'] = 1

In [ ]:
# Tuning 'subsample' and 'colsample_bytree' parameters
# Create list of possible params
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [ ]:
min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
# Update params dictionary
params['subsample'] = 0.9
params['colsample_bytree'] = 0.9

In [ ]:
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

In [ ]:
# Update parameters dictionary
params['eta'] = .05

In [ ]:
params

In [ ]:
# Train the model with the tuned parameters and use test data
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

In [ ]:
# Save model with best parameters
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [ ]:
# Create prediction
y_pred = best_model.predict(dtest)

In [ ]:
# Save model
best_model.save_model("xgboost_optimal_2020.model")

In [ ]:
# Calculate R squared and Adjusted R Square
import statsmodels.api as sm
result = sm.OLS(y_pred, y_test).fit()
print(result.rsquared, result.rsquared_adj)

In [ ]:
# Calculate Mean Squared Error
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import math
print(mean_squared_error(y_test, y_pred))
print(math.sqrt(mean_squared_error(y_test, y_pred)))
print(mean_squared_log_error(y_test, y_pred))

In [ ]:
# Calculate Mean Absolute Error(MAE)
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt

# extra step to allow graphviz to be found 
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/danny/.conda/envs/mlenv/lib/site-packages/graphviz'

xgb.plot_tree(model, num_trees=0)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()

In [ ]:
xgb.plot_importance(best_model)
plt.rcParams['figure.figsize'] = [5, 5]
plt.show()

In [ ]:
# # Code to load model for other datasets:
# loaded_model = xgb.Booster()
# loaded_model.load_model("my_model.model")
# # And use it for predictions.
# loaded_model.predict(dtest)